In [113]:
%run imports.ipynb

imports loaded.



In [15]:
print('> file management func:\n')

> file management func:



In [25]:
def get_content(directory):
    content = []
    for file in os.scandir(directory):
        if file.is_dir():
            content.append(file.path + '/')
        if file.is_file():
            content.append(file.path)
    return content
    
print('    get_content(directory)')

    get_content(directory)


In [8]:
def get_last_update(file_path):

    last_modified_timestamp = os.path.getmtime(file_path)
    last_modified_datetime = datetime.fromtimestamp(last_modified_timestamp)
    return last_modified_datetime.strftime("%d-%m-%Y")

print('    get_last_update(file_path)')

    get_last_update(file_path)


In [18]:
print('\n> fighters_stats_scrape func:\n')


> fighters_stats_scrape func:



In [53]:
def get_xml(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}):
    
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'xml')
    except:
        #print('Error on: ' + url) 
        return None
    
    return soup

print("    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.")

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.


In [97]:
def get_links(url):
    
    link_items =  get_xml(url=url).find_all('a',{'class':'b-link b-link_style_black'})
    links = []
    for item in link_items:
        links.append(item['href'])
    
    return links
   
print('    get_links(url)')

    get_links(url)


In [111]:
def get_fighter_data(url):
    
    xml =  get_xml(url=url)
    data = {}
    data['name'] = xml.find('h2').find('span',{'class':'b-content__title-highlight'}).text.strip()
    
    record_string = xml.find('h2').find('span',{'class':'b-content__title-record'}).text.strip()
    data['wins'] = int(record_string[record_string.find(':')+1:record_string.find('-')])
    data['losses'] = int(record_string.split('-')[1])
    data['draws'] = record_string.split('-')[2]
    
    
    stat_boxes = get_xml(url=url).find_all('ul')[0:3]
    for item in stat_boxes:
        for sub_item in item.contents:
            text = sub_item.text.strip().replace(" ", "").replace("\n", "")
            if len(text) > 0:
                key = text[:text.find(':')]
                value = text[text.find(':')+1:]
                data[key] = value
    
    data['last_update'] = get_current_date()
    data['last_fight'] = None
    return data

print('    get_fighter_data(url) # Retrive data as dictionery') 

    get_fighter_data(url) # Retrive data as dictionery


In [130]:
def get_fighter_data(url):
    
    xml =  get_xml(url=url)
    data = []
    data.append(xml.find('h2').find('span',{'class':'b-content__title-highlight'}).text.strip()) #name
    
    record_string = xml.find('h2').find('span',{'class':'b-content__title-record'}).text.strip()
    data.append(int(record_string[record_string.find(':')+1:record_string.find('-')])) #wins
    data.append(int(record_string.split('-')[1])) #losses
    data.append(record_string.split('-')[2]) #draws
    
    stat_boxes = xml.find_all('ul')[0:3]
    for item in stat_boxes:
        for sub_item in item.contents:
            text = sub_item.text.strip().replace(" ", "").replace("\n", "")
            if len(text) > 0:
                data.append(text[text.find(':')+1:])
    
    data.append(get_current_date()) #last_update
    
    date_table_location = 7
    try:
        date_string = xml.find_all('td')[date_table_location].find_all('p')[1].text.strip()
        last_match = datetime.strptime(date_string, '%b. %d, %Y').strftime('%d-%m-%Y')
    except:
        last_match = None

    data.append(last_match) #last_match
    return np.array(data)

print('    get_fighter_data(url) # Retrive data as list') 

    get_fighter_data(url) # Retrive data as list


In [114]:
def get_current_date():
    return datetime.today().strftime('%d-%m-%Y')
    
print('    get_current_date()')    

    get_current_date()


In [131]:
print('\n> matches_stats_web_scrape func:\n')


> matches_stats_web_scrape func:



In [155]:
def get_event_data(event_link):
    xml = get_xml(url=event_link)
    data_items = xml.find_all('td')
    row_length = 10
    
    row = []
    
    for i in range(row_length):
        row.append(data_items[i].find('p').text.strip())
        
    return row
    
    columns_items = xml.find('div',{'class':'b-fight-details'}).find_all('th')
    columns = []
    for item in columns_items:
        columns.append(item.text.strip())
    
    return columns

print('    get_match_link(event_link)')

    get_match_link(event_link)


In [46]:
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')

>>>>>


In [ ]:

def extract_fight_data(event_url):
    # Send a GET request to the event page
    response = requests.get(event_url)
    
    # Create a BeautifulSoup object from the response content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all the rows in the "FIGHTS" table
    table = soup.find('table', {'class': 'b-fight-details__table'})
    rows = table.find_all('tr', {'class': 'b-fight-details__table-row'})
    
    # Loop through each row and extract the relevant data
    fight_data = []
    for row in rows:
        # Find the columns in the row
        cols = row.find_all('td')
        
        # Extract the data from each column
        fighter1 = cols[1].find('a').text.strip()
        fighter2 = cols[3].find('a').text.strip()
        weight_class = cols[5].text.strip()
        method = cols[6].text.strip()
        round = cols[7].text.strip()
        time = cols[8].text.strip()
        
        # Add the data to the fight_data list
        fight_data.append({
            'fighter1': fighter1,
            'fighter2': fighter2,
            'weight_class': weight_class,
            'method': method,
            'round': round,
            'time': time
        })
    
    return fight_data

print('    extract_fight_data(event_url)')


In [28]:
def get_fight_columns(soup):
    
    column_items = soup.find_all('th',{'class':'b-fight-details__table-col'})
    columns = []
    for item in column_items:
        columns.append(item.text.strip())
          
    return columns
print('    get_fight_columns(soup)')

    get_columns(soup)


In [16]:
def get_fight_list(soup):
    
    fights_list = soup.find_all('tr',{'class':'b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click'})
    return fights_list

    links = fights_list.find('a')
    return links

print('    get_fight_list(soup)')

    get_fight_list(soup)


In [10]:
def get_event_date(soup):
    
    '''return event date'''
    
    item = soup.find('li',{'class':'b-list__box-list-item'})
    string = item.get_text().strip()
    start = len('DATE:')
    date = string[start:].strip()
    
    return datetime.strptime(date, "%B %d, %Y").strftime("%d-%m-%Y")

print('    get_event_date(soup)')

    get_event_date(soup)


In [12]:
def get_later_date(date1, date2):

    date1_obj = datetime.strptime(date1, "%d-%m-%Y")
    date2_obj = datetime.strptime(date2, "%d-%m-%Y")

    if date1_obj > date2_obj:
        return date1
    else:
        return date2
    
print('    get_later_date(date1, date2)')    

    get_later_date(date1, date2)
